## Packages

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

## Data

In [2]:
# load dataset
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
# train and test datasets are directly available, extract them accordingly
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']
# no validation sets so we have to split it on our own (in train data)

# store number of validation samples, test samples into dedicated variables
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples # .num_examples returns the dataset size
num_validation_samples = tf.cast(num_validation_samples, tf.int64) # cast as integer to prevent errors
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [3]:
# we want to scale our data so that the results are more numerically stable
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale) # .map() method allows us to apply a custom transformation (scale) to a given dataset
test_data = mnist_test.map(scale)

# shuffle the data
BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

# extract validation and train data
validation_data = shuffled_train_and_validation_data.take(num_validation_samples) # .take() method takes that many samples
train_data = shuffled_train_and_validation_data.skip(num_validation_samples) # .skip() method skips that many samples

In [4]:
# set batch size
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model


In [5]:
# outline model
input_size = 784
output_size = 10
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

In [6]:
# choose optimizer and loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
# training
NUM_EPOCHS = 5
model.fit(train_data, epochs=NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.4221 - accuracy: 0.8803 - val_loss: 0.2203 - val_accuracy: 0.9358 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 1s - loss: 0.1904 - accuracy: 0.9446 - val_loss: 0.1737 - val_accuracy: 0.9468 - 1s/epoch - 2ms/step
Epoch 3/5
540/540 - 1s - loss: 0.1465 - accuracy: 0.9572 - val_loss: 0.1391 - val_accuracy: 0.9590 - 1s/epoch - 2ms/step
Epoch 4/5
540/540 - 1s - loss: 0.1193 - accuracy: 0.9644 - val_loss: 0.1125 - val_accuracy: 0.9672 - 1s/epoch - 2ms/step
Epoch 5/5
540/540 - 1s - loss: 0.1003 - accuracy: 0.9704 - val_loss: 0.1037 - val_accuracy: 0.9683 - 1s/epoch - 2ms/step
